Access the libraries used for model building.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

Acces the dataset.

In [2]:
data = pd.read_csv('Final_Dataset1.csv')

print(data.head())

   Id Category                     Title  \
0   1     ayam          Ayam Woku Manado   
1   2     ayam  Ayam Goreng Tulang Lunak   
2   3     ayam          Ayam Cabai Kawin   
3   4     ayam               Ayam Geprek   
4   6     ayam           Nasi Bakar Ayam   

                                         Ingredients  \
0  1 Ekor Ayam Kampung (potong 12)--2 Buah Jeruk ...   
1  1 kg ayam (dipotong sesuai selera jangan kecil...   
2  1/4 kg ayam--3 buah cabai hijau besar--7 buah ...   
3  250 gr daging ayam (saya pakai fillet)--Secuku...   
4  1 piring nasi--¹/⁴ fillet ayam, potong kotak, ...   

                                               Steps  \
0  Cuci bersih ayam dan tiriskan. Lalu peras jeru...   
1  Haluskan bumbu2nya (BaPut, ketumbar, kemiri, k...   
2  Panaskan minyak di dalam wajan. Setelah minyak...   
3  Goreng ayam seperti ayam krispi--Ulek semua ba...   
4  Tumis bumbu halus, masukkan daun salam, serai-...   

                              URL  \
0          1_ayam_woku_m

Preparing data for text classification model training.

In [3]:
data['text'] = data['Title'] + ' ' + data['Ingredients'] + ' ' + data['Steps']
X = data['text']
y = data['Category']

label_encoder = tf.keras.preprocessing.text.Tokenizer()
label_encoder.fit_on_texts(y)
y_encoded = label_encoder.texts_to_sequences(y)
y_categorical = tf.keras.utils.to_categorical(y_encoded)

X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_length = max(len(x) for x in X_train_seq)
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=128),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y_categorical.shape[1], activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [5]:
model.fit(X_train_pad, y_train, epochs=5, batch_size=16, validation_data=(X_test_pad, y_test))

Epoch 1/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 8s 84ms/step - accuracy: 0.2893 - loss: 1.8873 - val_accuracy: 0.6681 - val_loss: 1.0796
Epoch 2/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - accuracy: 0.7787 - loss: 0.7678 - val_accuracy: 0.9447 - val_loss: 0.2500
Epoch 3/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.9709 - loss: 0.1552 - val_accuracy: 0.9660 - val_loss: 0.1382
Epoch 4/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 76ms/step - accuracy: 0.9943 - loss: 0.0513 - val_accuracy: 0.9532 - val_loss: 0.1450
Epoch 5/5
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 78ms/step - accuracy: 0.9909 - loss: 0.0320 - val_accuracy: 0.9617 - val_loss: 0.1287


In [6]:
loss, accuracy = model.evaluate(X_test_pad, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.9570 - loss: 0.1482
Test Accuracy: 0.96


In [ ]:
def recommend_recipe(CategoryUser , TempUser , TimeUser ):
    temp_condition = 1 if TempUser  <= 27 else 0
    
    if 6 <= TimeUser  < 10:
        type_condition = "Makan Pagi"
    elif 11 <= TimeUser  < 14:
        type_condition = "Makan Siang"
    elif 18 <= TimeUser  < 21:
        type_condition = "Makan Malam"
    else:
        type_condition = None

    filtered_data = data[(data['Category'] == CategoryUser ) & (data['Temp(cold)'] == temp_condition)]
    
    if type_condition:
        filtered_data = filtered_data[filtered_data['Type'].str.contains(type_condition)]
    
    filtered_data.loc[:, 'score'] = filtered_data['text'].apply(
        lambda x: model.predict(tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([x]), 
                                                                              maxlen=max_length, padding='post')
                                                                              ).max())
    
    recommendations = filtered_data[['Id', 'Title', 'Type', 'Temp(cold)', 'score']].sort_values(by='score', ascending=False)
    return recommendations.head(12)

In [ ]:
# Contoh penggunaan fungsi rekomendasi
CategoryUser  = "telur"
TempUser  = 30
TimeUser  = 12
recommended_recipes = recommend_recipe(CategoryUser , TempUser , TimeUser )
print(recommended_recipes)

In [ ]:
model.save('Recipe_Recomendation_Model_v2.h5')
print("Model telah disimpan dalam format .h5")